# Correlated Noise/Allan Variance Analysis

In [ ]:
import numpy as np
from tshirt.pipeline import phot_pipeline
from astropy.io import fits, ascii
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal as mvn
%matplotlib inline

<div class='alert alert-info'>
    <font size='3'><b>These cells contain instructions. Edit the paths below.</b></font>
</div>

In [ ]:
resid_path = 'resid2d.fits'
trimStart = 100 ## extra trimming to avoid spectrum edges
trimEnd = 1600

### Read in lightcurve residuals

In [ ]:
residHDU = fits.open(resid_path)
resid2D_orig = residHDU['RESID'].data[:,trimStart:trimEnd]
resid2D = resid2D_orig - np.median(resid2D_orig)
resid2D_err = residHDU["RESID ERR"].data[:,trimStart:trimEnd]

time = (residHDU["TIME"].data - np.min(residHDU["TIME"].data)) * 24. * 60. ## days to minutes

resid2D.shape

In [ ]:
overall_sigma = np.std(resid2D)
overall_sigma

### Simulate random data for comparison.
One that is IID (independent identically distributed) and one that is correlated

In [ ]:
np.random.seed(1)

sig_iid = overall_sigma
nwav = resid2D.shape[1]
nint = resid2D.shape[0]

meanval = 0.0

sim2D_iid = np.random.randn(nint,nwav) * sig_iid + meanval
sim2D_err = np.ones_like(sim2D_iid) * sig_iid

#sim_t = np.arange(nint) / float(nint) * 3600. * 4.
sim_t = time

size_cov = nwav
cor_tau = 100.
i2D, j2D = np.mgrid[:size_cov,:size_cov]
cov = np.exp(-(i2D - j2D)**2 / cor_tau**2) * sig_iid**2 * 0.1 + sig_iid**2 * np.eye(nwav)

sim2D_cov_wav = np.random.multivariate_normal(mean=np.zeros(size_cov) + meanval,cov=cov,size=nint)

size_cov = nint
i2D, j2D = np.mgrid[:size_cov,:size_cov]
cov = np.exp(-(i2D - j2D)**2 / cor_tau**2) * sig_iid**2 * 0.05 + sig_iid**2 * np.eye(nint)

sim2D_cov_tim = np.random.multivariate_normal(mean=np.zeros(size_cov) + meanval,cov=cov,size=nwav).T

In [ ]:
def show_dynamic(flux2D):
    fig, ax = plt.subplots()
    plt.imshow(flux2D,vmin=meanval-sig_iid * 0.2,vmax=meanval+sig_iid * 0.2)
    ax.set_xlabel("Wavelength (px)")
    ax.set_ylabel("Time (ints)")
show_dynamic(sim2D_iid)

In [ ]:
show_dynamic(sim2D_cov_wav)

In [ ]:
show_dynamic(sim2D_cov_tim)

In [ ]:
show_dynamic(resid2D)

## Wavelength binning

In [ ]:
def do_wavebins(flux2D,err2D,binSize):
    shape2D = flux2D.shape
    nints, nwav = shape2D[0], shape2D[1]
    
    nbins = int(np.floor(nwav / binSize ))

    flux_binned2D = np.zeros([nint,nbins])
    err_binned2D = np.zeros_like(flux_binned2D)
    for one_bin in np.arange(nbins):
        ind_st = one_bin * binSize
        ind_end = ind_st + binSize
        flux_binned2D[:,one_bin] = np.nansum(flux2D[:,ind_st:ind_end],axis=1) / binSize
        err_binned2D[:,one_bin] = np.sqrt(np.nansum(err2D[:,ind_st:ind_end]**2,axis=1)) / binSize
    
    resultDict = {}
    resultDict['nbins'] = nbins
    resultDict['flux_binned2D'] = flux_binned2D
    resultDict['std_binned'] = np.nanstd(flux_binned2D,axis=0)
    resultDict['theo_median'] = np.nanmedian(err_binned2D)#,axis=0)
    return resultDict

def wavebin(flux2D=sim2D_iid,err2D=sim2D_err):
    nwav = flux2D.shape[1]
    binpts_arr = 2**np.arange(11)

    binsize_list = []
    measured_list = []
    theoretical_list = []
    nbins_list = []

    for one_binSize in binpts_arr:
        resultDict = do_wavebins(flux2D,err2D,one_binSize)
        nbins_list.append(resultDict['nbins'])
        measured_list.append(resultDict['std_binned'])
        theoretical_list.append(resultDict['theo_median'])

    fig, ax = plt.subplots()
    theo_arr = np.array(theoretical_list) * 1e3
    ax.loglog(binpts_arr,theo_arr,label="Ideal IID")
    theo_with_floor = np.sqrt(theo_arr**2 + 0.1**2)
    ax.loglog(binpts_arr,theo_with_floor,label="With 100 ppm Floor")
    
    for ind,one_binSize in enumerate(binpts_arr):
        ax.plot(np.ones(nbins_list[ind]) * one_binSize,
                measured_list[ind] * 1e3,'o')
    ax.legend()
    ax.set_xlabel("N Wavebins")
    ax.set_ylabel("Error (ppt)")

In [ ]:
wavebin(sim2D_iid)

In [ ]:
wavebin(sim2D_cov_tim)

In [ ]:
wavebin(sim2D_cov_wav)

In [ ]:
wavebin(resid2D,err2D=resid2D_err)

Time Binning

In [ ]:
help(phot_pipeline.allan_variance)

In [ ]:
def timebin(flux2D,err2D=sim2D_err):
    resultDict = do_wavebins(flux2D,err2D,64)
    phot_pipeline.allan_variance(sim_t,resultDict['flux_binned2D'][:,0],
                                 yerr=resultDict['theo_median'],showPlot=True,
                                 binMin=5)

In [ ]:
timebin(sim2D_iid)

In [ ]:
timebin(sim2D_cov_wav)

In [ ]:
timebin(sim2D_cov_tim)

In [ ]:
timebin(resid2D,err2D=resid2D_err)